In [1]:
import numpy as np
import warnings
from utils import *
from calibrators import * 
import pickle
warnings.filterwarnings("ignore")
import torch
import sys
sys.path.append('../')
import time
import copy
from SLURM.sklearn_config import *
import time
import tensorflow as tf
from skimage import color
from skimage import io

# check if the reduction in size can reduce number of diffrent example

In [66]:
datasets = ['MNIST','Fashion','SignLanguage']#,'CIFAR_RGB','GTSRB_RGB']
models = ['RF','GB','pytorch']
for pool_size in range(1,5):
    for dataset_name in datasets:
        for model_name in models:
            flag=[0]*10
            for shuffle in range(10):
                model_info= load_model(dataset_name, model_name, shuffle)
                data=model_info.data
                polling = torch.nn.AvgPool2d(pool_size)
                pixels=int(sqrt(data.X_train.shape[1]))
                train=polling(torch.tensor(model_info.data.X_train.reshape((len(data.X_train),pixels,pixels)))).reshape(len(data.X_train),-1)
                if len(set(train)) != len(train):
                    print(shuffle)
                    flag[shuffle]=1
            if flag!=[0]*10:
                print(f'{dataset_name}-{model_name}-{pool_size}-Avg-find colision')
                print(flag)
                
datasets = ['MNIST','Fashion','SignLanguage']#,'CIFAR_RGB','GTSRB_RGB']
models = ['RF','GB','pytorch']
for pool_size in range(1,5):
    for dataset_name in datasets:
        for model_name in models:
            flag=[0]*10
            for shuffle in range(10):
                model_info= load_model(dataset_name, model_name, shuffle)
                data=model_info.data
                polling = torch.nn.MaxPool2d(pool_size)
                pixels=int(sqrt(data.X_train.shape[1]))
                train=polling(torch.tensor(model_info.data.X_train.reshape((len(data.X_train),pixels,pixels)))).reshape(len(data.X_train),-1)
                if len(set(train)) != len(train):
                    print(shuffle)
                    flag[shuffle]=1
            if flag!=[0]*10:
                print(f'{dataset_name}-{model_name}-{pool_size}-max-find colision')
                print(flag)
                                

In [67]:
datasets = ['CIFAR_RGB','GTSRB_RGB']
models = ['RF','GB','pytorch']
for pool_size in range(1,5):
    for dataset_name in datasets:
        for model_name in models:
            flag=[0]*10
            for shuffle in range(10):
                model_info= load_model(dataset_name, model_name, shuffle)
                data=model_info.data
                polling = torch.nn.AvgPool2d(pool_size)
                
                pixels=int(sqrt(model_info.data.X_train.shape[1]/3))
                trainX=model_info.data.X_train.reshape(len(model_info.data.X_train),pixels,pixels,3)
                imgGray_train = color.rgb2gray(trainX)
                train=polling(torch.tensor(imgGray_train)).reshape(len(model_info.data.X_train),-1)               
                if len(set(train)) != len(train):
                    print(shuffle)
                    flag[shuffle]=1
            if flag!=[0]*10:
                print(f'{dataset_name}-{model_name}-{pool_size}-Avg-find colision')
                print(flag)  
datasets = ['CIFAR_RGB','GTSRB_RGB']
models = ['RF','GB','pytorch']
for pool_size in range(1,5):
    for dataset_name in datasets:
        for model_name in models:
            flag=[0]*10
            for shuffle in range(10):
                model_info= load_model(dataset_name, model_name, shuffle)
                data=model_info.data
                polling = torch.nn.MaxPool2d(pool_size)
                
                pixels=int(sqrt(model_info.data.X_train.shape[1]/3))
                trainX=model_info.data.X_train.reshape(len(model_info.data.X_train),pixels,pixels,3)
                imgGray_train = color.rgb2gray(trainX)
                train=polling(torch.tensor(imgGray_train)).reshape(len(model_info.data.X_train),-1)               
                if len(set(train)) != len(train):
                    print(shuffle)
                    flag[shuffle]=1
            if flag!=[0]*10:
                print(f'{dataset_name}-{model_name}-{pool_size}-Max-find colision')
                print(flag) 

# check the memory size to ship the NN models or the data itself:

## NN model

In [3]:
datasets = ['MNIST','Fashion','SignLanguage','CIFAR_RGB','GTSRB_RGB']
mem_dict={}

for pool_size in range(1,5):
    data_dict={}
    for dataset_name in datasets:
        all_mem=[]
        for shuffle in range(10):                
            mem_same,mem_other=check_nn_memory(dataset_name,'RF',shuffle,pool_type='Avg',pool_size=pool_size)
            all_mem.append(sum(mem_same+mem_other))

        data_dict[f'{dataset_name}']=np.array(all_mem).mean()
    mem_dict[pool_size]=data_dict
df_memory=pd.DataFrame(mem_dict)
df_memory.to_csv("memory_df.csv")
df_memory

,1,2,3,4
MNIST,2634.250300,658.570260,272.170260,164.650260
Fashion,2634.250300,658.570260,272.170260,164.650260
SignLanguage,3127.243920,781.829424,323.117424,195.475824
CIFAR_RGB,2949.130300,737.290300,288.010260,184.330260
GTSRB_RGB,9629.533068,2407.416296,1069.987296,524.316241


In [4]:
datasets = ['MNIST','Fashion','SignLanguage','CIFAR_RGB','GTSRB_RGB']
mem_dict={}

for pool_size in range(1,5):
    data_dict={}
    for dataset_name in datasets:
        all_mem=[]
        for shuffle in range(10):                
            mem_same,mem_other=check_nn_memory(dataset_name,'RF',shuffle,pool_type='Max',pool_size=pool_size)
            all_mem.append(sum(mem_same+mem_other))

        data_dict[f'{dataset_name}']=np.array(all_mem).mean()
    mem_dict[pool_size]=data_dict
df_memory_max=pd.DataFrame(mem_dict)
df_memory_max.to_csv("memory_max_df.csv")
df_memory_max

,1,2,3,4
MNIST,2634.250300,658.570260,272.170260,164.650260
Fashion,2634.250300,658.570260,272.170260,164.650260
SignLanguage,3127.243920,781.829424,323.117424,195.475824
CIFAR_RGB,2949.130300,737.290300,288.010260,184.330260
GTSRB_RGB,9629.533068,2407.416296,1069.987296,524.316241


## Data itself

In [8]:
datasets = ['MNIST','Fashion','SignLanguage','CIFAR_RGB','GTSRB_RGB']
model_name='RF'
mem_dict={}

pool_type='Avg'
for pool_size in range(1,5):
    data_dict={}
    for dataset_name in datasets:
        mem_list=[]
        for shuffle in range(10):                
            model_info= load_model(dataset_name, model_name, shuffle)
            trainX=model_info.data.X_train
            train_y=model_info.data.y_train
            
            if pool_type=='Avg':
                polling = torch.nn.AvgPool2d(pool_size)
            elif pool_type=='Max':
                polling = torch.nn.MaxPool2d(pool_size)
                
            RGB='RGB' in dataset_name     
            if RGB==False:
                pixels=int(sqrt(trainX.shape[1]))
                trainX=polling(torch.tensor(trainX.reshape((len(trainX),pixels,pixels)))).reshape(len(trainX),-1)

            else:
                pixels=int(sqrt(trainX.shape[1]/3))
                train=trainX.reshape(len(trainX),pixels,pixels,3)
                imgGray_train = color.rgb2gray(train)
                trainX=polling(torch.tensor(imgGray_train)).reshape(len(trainX),-1)
            file=f'./NN_models/{dataset_name}_{model_name}_shuffle{shuffle}_{pool_size}{pool_type}'
            np.save(file+f'trainX.npy',trainX)
            np.save(file+f'trainy.npy',train_y)
            size=os.path.getsize(file+f'trainX.npy')+os.path.getsize(file+f'trainy.npy')
            os.remove(file+f'trainX.npy')
            os.remove(file+f'trainy.npy')
            mem_list.append(size/10**6)
        data_dict[f'{dataset_name}']=np.array(mem_list).mean()
    mem_dict[pool_size]=data_dict
df_memory_data=pd.DataFrame(mem_dict)
df_memory_data.to_csv("memory_data_df_avg.csv")
df_memory_data

,1,2,3,4
MNIST,263.466256,65.898256,27.258256,16.506256
Fashion,263.466256,65.898256,27.258256,16.506256
SignLanguage,130.467256,32.741656,13.628656,8.310256
CIFAR_RGB,294.948256,73.764256,28.836256,18.468256
GTSRB_RGB,224.190680,56.234480,25.131480,12.441456


In [9]:
datasets = ['MNIST','Fashion','SignLanguage','CIFAR_RGB','GTSRB_RGB']
model_name='RF'
mem_dict={}

pool_type='Max'
for pool_size in range(1,5):
    data_dict={}
    for dataset_name in datasets:
        mem_list=[]
        for shuffle in range(10):                
            model_info= load_model(dataset_name, model_name, shuffle)
            trainX=model_info.data.X_train
            train_y=model_info.data.y_train
            
            if pool_type=='Avg':
                polling = torch.nn.AvgPool2d(pool_size)
            elif pool_type=='Max':
                polling = torch.nn.MaxPool2d(pool_size)
                
            RGB='RGB' in dataset_name     
            if RGB==False:
                pixels=int(sqrt(trainX.shape[1]))
                trainX=polling(torch.tensor(trainX.reshape((len(trainX),pixels,pixels)))).reshape(len(trainX),-1)

            else:
                pixels=int(sqrt(trainX.shape[1]/3))
                train=trainX.reshape(len(trainX),pixels,pixels,3)
                imgGray_train = color.rgb2gray(train)
                trainX=polling(torch.tensor(imgGray_train)).reshape(len(trainX),-1)
            file=f'./NN_models/{dataset_name}_{model_name}_shuffle{shuffle}_{pool_size}{pool_type}'
            np.save(file+f'trainX.npy',trainX)
            np.save(file+f'trainy.npy',train_y)
            size=os.path.getsize(file+f'trainX.npy')+os.path.getsize(file+f'trainy.npy')
            os.remove(file+f'trainX.npy')
            os.remove(file+f'trainy.npy')
            mem_list.append(size/10**6)
        data_dict[f'{dataset_name}']=np.array(mem_list).mean()
    mem_dict[pool_size]=data_dict
df_memory_data=pd.DataFrame(mem_dict)
df_memory_data.to_csv("memory_data_df_max.csv")
df_memory_data

,1,2,3,4
MNIST,263.466256,65.898256,27.258256,16.506256
Fashion,263.466256,65.898256,27.258256,16.506256
SignLanguage,130.467256,32.741656,13.628656,8.310256
CIFAR_RGB,294.948256,73.764256,28.836256,18.468256
GTSRB_RGB,224.190680,56.234480,25.131480,12.441456
